# Testing Azure Functions

Testing Azure HTTP functions is easy because you can just use `requests` to ping the function. However EventHub functions are more difficult because you have to observe them happening in the context of the EventHub ecosystem. This usually means that you have to deploy and wait for your event to get triggered. 

Testing Azure Functions can be easy if you design your process right. The trick is to separate the Azure Function logic from the logic of your application. 

Like this:
```python
@app.function_name(name="myFunction")
@app.event_hub_message_trigger(arg_name="event",
                               event_hub_name=EVENT_HUB_NAME,
                               connection="EVENT_HUB_CONNECTION_STR")
def AZ_myFunction(event: func.EventHubEvent):
    eh_producer = EventHubProducerClient.from_connection_string(EVENT_HUB_CONNECTION_STR, eventhub_name=EVENT_HUB_NAME)
    credential = DefaultAzureCredential() 
    message = ast.literal_eval(event.get_body().decode('utf-8'))
    # proccessing messages is removed so that I can test it locally. 
    outgoing_messages = local_myFunction(message)
    if len(outgoing_messages)>0:
        logging.info(f"myFunction produced {len(outgoing_messages)} outgoing messages")
        send_to_eventhub(outgoing_messages, eh_producer)
        logging.info(f"Additional messages sent to EH. ")

def local_myFunction(message):
    # TODO: My business process that I can test locally
    outgoing_messages = []
    return outgoing_messages
```

This way I can deploy to the cloud, but I don't have to wait to see the results. I can deploy them locally and test my local business process. 

### A demonstration

In [1]:
import os
import  ssl, asyncio
import nest_asyncio
import pandas as pd

# moving back to the root, but idempotent in case we are already there
if 'function_app.py' not in os.listdir():
    os.chdir('..')
print([f for f in os.listdir() if f == 'function_app.py'])

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.info("logs showing as print")



ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

INFO:root:logs showing as print


['function_app.py']


In [2]:
import function_app as f

executing local windows deployment
something wrong with your query: <class 'Exception'>


## Generating the Messages to be Tested

Each Azure function has both a python function that is routed to the az function app, and a separate function that does the business logic of my game. This allows me to test locally.

| AZ Func Name | AZ Function | Relevant Local Function(args) | Description |
|----------|----------|----------|----------|
| actionResolverTimer | action_resolver | `process_action_messages()` | Queries open jobs and generates EventHub Messages to resolve them  |
| factionBuildingTimer | faction_building_resolver | `get_structure_messages()` | Structures that have ongoing effects |
| resolveActionEvents | resolve_action_event | `process_action_event_message(message)` | Takes individual event messages and resolves them |
| ututimer | utu_timer | `increment_timer()` | Increments the galatic timer |

In [7]:
action_messages = f.process_action_messages()

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: health requirement 0.7
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: healthy_pops_query 0
INFO:root:EXOADMIN: No pops that meet the pop_health_requirement
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: Total jobs: 1
INFO:root:EXOADMIN: job instance of ACTION created: {'action': {'type': 'construction', 'comment': 'constructing a Farmland', 'effort': '1', 'applies_to': 'pop', 'owned_by': 'pop', 'building': 'farmland', 'created_at': '1000', 'to_build': '{type: farmland, label: building, name: Farmland, description: Generates organic foodstuffs, populations will consume food before consuming natural resources, planet_requirements: {isHabitable: true}, faction_augments: {wealth: -1}, renews_faction_resource: {grains: 10}, owned_by: pop, effort: 1}', 'objid': '8329428173860', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'action'}, 'job': {'status': 'pending', 'userguid': '

In [9]:
pd.DataFrame(action_messages)

,agent,action,job
0,"{'isIdle': 'false', 'name': 'Ciu Damlantia', '...","{'type': 'construction', 'comment': 'construct...","{'status': 'pending', 'userguid': '8d5b667f-b2..."
1,"{'objid': '4975085035335', 'consumes': ['organ...",consume,NaN
2,"{'objid': '8510105182256', 'consumes': ['organ...",consume,NaN
3,"{'objid': '4558198014921', 'consumes': ['organ...",consume,NaN
4,"{'objid': '4966734219567', 'consumes': ['organ...",consume,NaN
5,"{'objid': '6238436631717', 'consumes': ['organ...",consume,NaN
6,"{'objid': '3449618203493', 'consumes': ['organ...",consume,NaN
7,"{'objid': '3069640564489', 'consumes': ['organ...",consume,NaN
8,"{'replenish_rate': '10', 'volume': 1084.0, 'na...",renew,NaN


## Resolving those messages

In [10]:
f.process_action_event_message(action_messages[0])

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: processing message: {'type': 'construction', 'comment': 'constructing a Farmland', 'effort': '1', 'applies_to': 'pop', 'owned_by': 'pop', 'building': 'farmland', 'created_at': '1000', 'to_build': '{type: farmland, label: building, name: Farmland, description: Generates organic foodstuffs, populations will consume food before consuming natural resources, planet_requirements: {isHabitable: true}, faction_augments: {wealth: -1}, renews_faction_resource: {grains: 10}, owned_by: pop, effort: 1}', 'objid': '8329428173860', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'action'} at UTU:< time at UTU:1002 >
INFO:root:EXOADMIN: job instance of ACTION created: {'agent': {'isIdle': 'false', 'name': 'Ciu Damlantia', 'objid': '3069640564489', 'conformity': '0.397', 'literacy': '0.658', 'aggression': '0.399', 'constitution': '0.465', 'health': '0.7', 'isIn': '7327160462281', 'industry': '0.43200000000000005', 'wealth':

[]

In [11]:
f.process_action_event_message(action_messages[1])

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: processing message: consume at UTU:< time at UTU:1002 >
INFO:root:EXOADMIN: Processing reduce_location_resource for: {'objid': '4975085035335', 'consumes': ['organics'], 'effuses': ['organic waste', 'plastics']}
INFO:root:EXOADMIN: resource_query query: 
    g.V().has('objid','4975085035335').out('inhabits').out('has').has('objtype','resource').has('name','organics').valuemap()
    
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: resources 6367214839613 consumed by 4975085035335: reduced by 2, 1084.0-> 1082.0
INFO:root:EXOADMIN: patch_resource_query: 
        g.V().has('objid','4975085035335')
            .out('inhabits')
            .out('has').has('label','resource')
            .has('name','organics')
            .property('volume',1082.0)
    
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: agent: 4975085035335 consumed resource: 6367214839613. 1084.0->1082.0
INFO:root:EXOADMIN:      

[]

## Incrementing the timer
Just for testing purposes. Some jobs can only be done when ready. 


In [6]:
f.increment_timer()

DEBUG:asyncio:Using selector: SelectSelector
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: UTU was updated, result: currentTime was updated from:1001 to: 1002 at: < time at UTU:1001 >
